In [1]:
import os
import requests
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import Select
import csv

In [3]:
def set_up_parkrun_dir():
    if not os.path.exists('./parkrun'):
        os.mkdir('./parkrun')

def scrape_personal_results(driver, athelete_number):
    set_up_parkrun_dir()
    all_results_url = 'https://www.parkrun.org.uk/results/athleteeventresultshistory/?athleteNumber={}&eventNumber=0'
    driver.get(all_results_url.format(athelete_number))
    run_rows = driver.find_elements_by_xpath('/html/body/div[1]/div[3]/div[1]/div/div[3]/table/tbody/tr[*]')
    with open('./parkrun/personal_results.csv', 'w+') as personal_results:    
        for row in run_rows:
            data = {
                'event': row.find_element_by_xpath('td[1]/a').text,
                'event_url': row.find_element_by_xpath('td[1]/a').get_attribute('href'),
                'run_date': row.find_element_by_xpath('td[2]/a').text,
                'run_date_url': row.find_element_by_xpath('td[2]/a').get_attribute('href'),
                'run_number': row.find_element_by_xpath('td[3]/a').text,
                'run_number_url': row.find_element_by_xpath('td[3]/a').get_attribute('href'),
                'pos': row.find_element_by_xpath('td[4]').text,
                'time': row.find_element_by_xpath('td[5]').text,
                'age_grade': row.find_element_by_xpath('td[6]').text,
                'pb': row.find_element_by_xpath('td[7]').text,
            }    
            personal_results.write('{event},{event_url},{run_date},{run_date_url},{run_number},{run_number_url},{pos},{time},{age_grade},{pb}\n'.format(**data))


In [10]:
driver = webdriver.Firefox()
scrape_personal_results(driver,'<PRNUMBER>')

In [11]:
def set_up_parkrun_race_results_dir():
    if not os.path.exists('./parkrun/race_results'):
        os.mkdir('./parkrun/race_results')
        
def close_search_tip_modal(driver):
    try:
        results_modal = driver.find_element_by_class_name('Results-modal-close')
        if results_modal:
            results_modal.click()
    except ElementNotInteractableException:
        pass
    
def scrape_all_race_results(driver):
    set_up_parkrun_race_results_dir()
    with open('./parkrun/personal_results.csv', 'r') as personal_results:
        personal_results_reader = csv.reader(personal_results)
        for result in personal_results_reader:
            race_result_filename = '{}_{}.csv'.format(result[0].lower(), result[4])
            with open('./parkrun/race_results/{}'.format(race_result_filename), 'w+') as race_result:
                race_result_writer = csv.writer(race_result)
                driver.get(result[3])
                close_search_tip_modal(driver)
                select = Select(driver.find_element_by_name('display'))
                select.select_by_visible_text('Detailed')
                rows = driver.find_elements_by_xpath('/html/body/div[1]/div[3]/div[1]/div/div[1]/table/tbody/tr[*]')
                for row in rows:
                    data = []
                    # position
                    data.append(row.find_element_by_xpath('td[1]').text)
                    # Name
                    try:
                        data.append(row.find_element_by_xpath('td[2]/div').text)
                    except NoSuchElementException:
                        data.append('')
                    # Name details
                    try:
                        data.append(row.find_element_by_xpath('td[2]/div[2]').text)
                    except NoSuchElementException:
                        data.append('')
                    # Gender
                    try:
                        data.append(row.find_element_by_xpath('td[3]/div').text)
                    except NoSuchElementException:
                        data.append('')
                    # Gender details
                    try:
                        data.append(row.find_element_by_xpath('td[3]/div[2]').text)
                    except NoSuchElementException:
                        data.append('')
                    # Age Group
                    try:
                        data.append(row.find_element_by_xpath('td[4]/div').text)
                    except NoSuchElementException:
                        data.append('')
                    # Age Group etail
                    try:
                        data.append(row.find_element_by_xpath('td[4]/div[2]').text)
                    except NoSuchElementException:
                        data.append('')
                    # Club
                    try:
                        data.append(row.find_element_by_xpath('td[5]/div').text)  
                    except NoSuchElementException:
                        data.append('')
                    # Time
                    try: 
                        data.append(row.find_element_by_xpath('td[6]/div[1]').text)
                    except NoSuchElementException:
                        data.append('')
                    # Time detail
                    try: 
                        data.append(row.find_element_by_xpath('td[6]/div[2]').text)
                    except NoSuchElementException:
                        data.append('')  
                    race_result_writer.writerow(data)
        

In [12]:
scrape_all_race_results(driver)